# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os
from pathlib import Path
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import json
from textwrap import indent
import nbformat


In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'gemma3'
api = '123...'

In [ ]:
# set up environment
load_dotenv(override=True)

In [ ]:
system_prompt = """
You are an expert code reviewer. 
you are provided with the python or python notebook files. 
you will review the code and suggest the areas where the code can be made more efficient.
you will also review the industry best practises for the code suggest any new edits to be incorporated for the efficient code 
you will also check for any comments and suggest if there is any improvement. check for any hardcoded values like passwords, or api keys and flag it
only the files with extension .py or .ipynb will be considered ignore the rest.
you will respond the output in the Markdown format for example: 
## test.py
- please explain what this function def func does
- Please dont hard code any passwords
"""

In [ ]:

def extract_code_fron_ipynb(filePath):
    with open(filePath,'r',encoding="utf-8") as f:
        nb = nbformat.read(f, as_version=4)
    
    return "\n\n".join([c.source for c in nb.cells if c.cell_type=='code'])

def get_file_content(filePath):
    suffix = Path(filePath).suffix

    if suffix == '.py':
        return Path(filePath).read_text
    elif suffix == '.ipynb':
        return extract_code_fron_ipynb(filePath)

def get_list_files(filePath):
    path = Path(filePath)
    files = {
        f.name: get_file_content(str(path / f.name)) for f in path.iterdir() if f.suffix in ['.py','.ipynb']
    }
    return files
    

In [ ]:
filePath = '/Users/mia/myprojects/uv_projects/llm_engineering/week1'
filesList = get_list_files(filePath)

In [ ]:
# here is the question; type over this to ask something new

question = f"""
Please explain what this code does and why:
the name and code is listed as below json file 
{filesList}
"""

question

In [ ]:
messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]

In [ ]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()
stream = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=messages,
    stream=True
)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
# Get Llama 3.2 to answer

llama=OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
llama_stream = llama.chat.completions.create(
    model=MODEL_LLAMA,
    messages=messages
)
response = llama_stream.choices[0].message.content
print(response)